In [1]:
from sedona.register import SedonaRegistrator
from pyspark.sql import SparkSession
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

from shapely.geometry import Point

In [2]:
spark = SparkSession.\
    builder.\
    appName("apache-sedona").\
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.postgresql:postgresql:42.2.23,org.datasyslab:geotools-wrapper:geotools-24.0") . \
    master("local[*]").\
    getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark-3.0.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ba00300-c4d9-4a46-b351-7c16eac14f81;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.14.3 in central
	found com.fasterxml.jackson.core#jackson-databind;2.10.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.10.0 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.10.0.pr1 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.0.0-i

# Register sedona functions

In [3]:
SedonaRegistrator.registerAll(spark)

True

# Creating spatial data frame

In [4]:
data = [
    [1, Point(21, 52)],
    [2, Point(45, 32)]
]

In [16]:
spatial_df = spark.createDataFrame(data).\
    selectExpr("_1 AS id", "ST_AsText(_2) AS geom")

In [17]:
spatial_df.show()

+---+-------------+
| id|         geom|
+---+-------------+
|  1|POINT (21 52)|
|  2|POINT (45 32)|
+---+-------------+



In [18]:
spatial_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- geom: string (nullable = false)



# Writing Spatial data frame to postgis

In [19]:
spatial_df.write .\
  format("jdbc") .\
  option("url", "jdbc:postgresql://postgis:5432/sedona") .\
  option("dbtable", "spatial_points") .\
  option("user", "sedona") .\
  option("password", "sedona") .\
  option("driver", "org.postgresql.Driver") .\
  save()

# Reading

In [24]:
spark.read .\
  format("jdbc") .\
  option("url", "jdbc:postgresql://postgis:5432/sedona") .\
  option("dbtable", "spatial_points") .\
  option("user", "sedona") .\
  option("password", "sedona") .\
  option("driver", "org.postgresql.Driver") .\
  load().\
  show()

+---+-------------+
| id|         geom|
+---+-------------+
|  1|POINT (21 52)|
|  2|POINT (45 32)|
+---+-------------+

